In [2]:
import kaggle 
!kaggle datasets download -d taeefnajib/used-car-price-prediction-dataset

Dataset URL: https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset
License(s): Attribution 4.0 International (CC BY 4.0)




  0%|          | 0.00/109k [00:00<?, ?B/s]
100%|██████████| 109k/109k [00:00<00:00, 160kB/s]
100%|██████████| 109k/109k [00:00<00:00, 160kB/s]


In [3]:
import zipfile
zip = zipfile.ZipFile('used-car-price-prediction-dataset.zip')
zip.extractall()
zip.close()

# Import Lib

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [48]:
df = pd.read_csv('used_cars.csv')

**Brand & Model:** Identify the brand or company name along with the specific model of each vehicle.  

**Model Year:** Discover the manufacturing year of the vehicles, crucial for assessing depreciation and technology advancements.  

**Mileage:** Obtain the mileage of each vehicle, a key indicator of wear and tear and potential maintenance requirements.  

**Fuel Type:** Learn about the type of fuel the vehicles run on, whether it's gasoline, diesel, electric, or hybrid.  

**Engine Type:** Understand the engine specifications, shedding light on performance and efficiency.  

**Transmission:** Determine the transmission type, whether automatic, manual, or another variant.  

**Exterior & Interior Colors:** Explore the aesthetic aspects of the vehicles, including exterior and interior color options.  

**Accident History:** Discover whether a vehicle has a prior history of accidents or damage, crucial for informed decision-making.  

**Clean Title:** Evaluate the availability of a clean title, which can impact the vehicle's resale value and legal status.  

**Price:** Access the listed prices for each vehicle, aiding in price comparison and budgeting.  

In [49]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


# Data Cleaning

In [41]:
df['price'] = df['price'].str.extract('(\d+,\d+)', expand=True)
df['price'] = df['price'].str.replace(',', '').astype(int)

In [43]:
df['milage'] =df['milage'].str.replace(' mi.', '')
df['milage'] = df['milage'].str.replace(',', '').astype(int)

In [53]:
df['fuel_type'].unique()

array(['E85 Flex Fuel', 'Gasoline', 'Hybrid', nan, 'Diesel',
       'Plug-In Hybrid', '–', 'not supported'], dtype=object)

## Datatype

In [45]:
df.isnull().sum()

brand             0
model             0
model_year        0
milage            0
fuel_type       170
engine            0
transmission      0
ext_col           0
int_col           0
accident        113
clean_title     596
price             0
dtype: int64

In [7]:
df['fuel_type'].unique()

array(['E85 Flex Fuel', 'Gasoline', 'Hybrid', nan, 'Diesel',
       'Plug-In Hybrid', '–', 'not supported'], dtype=object)

In [8]:
df.groupby('fuel_type')['fuel_type'].count()

fuel_type
Diesel             116
E85 Flex Fuel      139
Gasoline          3309
Hybrid             194
Plug-In Hybrid      34
not supported        2
–                   45
Name: fuel_type, dtype: int64

In [9]:
df['brand'].nunique()

57

In [10]:
df.groupby('brand')['price'].mean().sort_values(ascending=False).head(10)

brand
Rolls-Royce    370992.727273
Lamborghini    291233.846154
Ferrari        243790.666667
McLaren        213457.500000
Bentley        137553.545455
Aston          115199.555556
Lucid          101966.333333
Rivian          93138.176471
Porsche         80804.029851
Maybach         64250.000000
Name: price, dtype: float64

In [11]:
df[df['brand'] == 'Bugatti']

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
229,Bugatti,Veyron 16.4 Grand Sport,2011,6330,Gasoline,8.0L W16 64V GDI DOHC Twin Turbo,7-Speed Automatic with Auto-Shift,White,White,None reported,Yes,1950


In [12]:
df['brand'].unique()

array(['Ford', 'Hyundai', 'Lexus', 'INFINITI', 'Audi', 'Acura', 'BMW',
       'Tesla', 'Land', 'Aston', 'Toyota', 'Lincoln', 'Jaguar',
       'Mercedes-Benz', 'Dodge', 'Nissan', 'Genesis', 'Chevrolet', 'Kia',
       'Jeep', 'Bentley', 'Honda', 'Lucid', 'MINI', 'Porsche', 'Hummer',
       'Chrysler', 'Volvo', 'Cadillac', 'Lamborghini', 'Maserati',
       'Volkswagen', 'Subaru', 'Rivian', 'GMC', 'RAM', 'Alfa', 'Ferrari',
       'Scion', 'Mitsubishi', 'Mazda', 'Saturn', 'Bugatti', 'Polestar',
       'Rolls-Royce', 'McLaren', 'Buick', 'Lotus', 'Pontiac', 'FIAT',
       'Karma', 'Saab', 'Mercury', 'Plymouth', 'smart', 'Maybach',
       'Suzuki'], dtype=object)

In [13]:
df['transmission_type'] = df['transmission'].apply(lambda x: 'Automatic' if 'a/t' in x.lower() or 'automatic' in x.lower() else 'Manual')

In [14]:
df.sample(5)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,transmission_type
3357,Chevrolet,Corvette Stingray w/3LT,2022,5400,Gasoline,490.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,None reported,Yes,89900,Automatic
3831,Volkswagen,Jetta 1.4T R-Line,2019,55234,Gasoline,1.4L I4 16V GDI DOHC Turbo,8-Speed Automatic,Platinum Gray Metallic,Black / Gray,None reported,NaN,19290,Automatic
168,GMC,Terrain SLE,2022,27352,Gasoline,1.5L I4 16V GDI DOHC Turbo,9-Speed Automatic,Summit White,Jet Black,None reported,NaN,27785,Automatic
910,Mercedes-Benz,S-Class S 63 AMG,2011,60000,Gasoline,536.0HP 5.5L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Black,At least 1 accident or damage reported,Yes,37000,Manual
1859,BMW,M4 Competition xDrive,2022,15882,Gasoline,503.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Red,Silver,None reported,Yes,84995,Automatic


In [15]:
from datetime import datetime
df['model_age'] =datetime.now().year - df['model_year']

In [16]:
df['ext_col'].nunique()

319

In [17]:
df.sample(5)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,transmission_type,model_age
988,Land,Rover Range Rover Sport First Edition,2023,3534,Hybrid,4.4 Liter,Automatic,Red,Black,None reported,NaN,143998,Automatic,1
3312,BMW,330 i,2019,42200,Gasoline,255.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,None reported,Yes,27500,Automatic,5
3187,Toyota,Tundra SR5,2016,119000,Gasoline,381.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Red,Black,None reported,Yes,29950,Automatic,8
1629,Ford,Escape XLT,2007,133300,Gasoline,153.0HP 2.3L 4 Cylinder Engine Gasoline Fuel,4-Speed A/T,Silver,Silver,None reported,Yes,3000,Automatic,17
3468,BMW,i3 94 Ah,2017,43000,NaN,170.0HP 0.65L Electric Motor Electric Fuel System,1-Speed A/T,Black,–,None reported,Yes,16500,Automatic,7


In [24]:
list(filter(lambda x: 'A/T' in x, df['transmission'].unique()))

['6-Speed A/T',
 '7-Speed A/T',
 'A/T',
 '8-Speed A/T',
 '10-Speed A/T',
 '9-Speed A/T',
 '5-Speed A/T',
 '1-Speed A/T',
 '4-Speed A/T',
 '8-SPEED A/T',
 '2-Speed A/T']

In [26]:
df['transmission'].unique()

array(['6-Speed A/T', '8-Speed Automatic', 'Automatic', '7-Speed A/T',
       'F', 'A/T', '8-Speed A/T', 'Transmission w/Dual Shift Mode',
       '9-Speed Automatic', '6-Speed M/T', '10-Speed A/T', '9-Speed A/T',
       '5-Speed A/T', '1-Speed A/T', 'Automatic CVT',
       '7-Speed Automatic with Auto-Shift', 'CVT-F', 'M/T',
       '6-Speed Automatic with Auto-Shift', '10-Speed Automatic',
       'CVT Transmission', '4-Speed A/T', '6-Speed Automatic',
       '4-Speed Automatic', '7-Speed M/T',
       '8-Speed Automatic with Auto-Shift', '5-Speed Automatic',
       '8-SPEED AT', '1-Speed Automatic', '5-Speed M/T', 'Manual',
       '6-Speed Manual', 'Variable', '2', '7-Speed Manual',
       'Automatic, 9-Spd 9G-Tronic', 'Automatic, 8-Spd',
       'Automatic, 8-Spd Sport w/Sport & Manual Modes', '–',
       'Auto, 6-Spd w/CmdShft', 'Transmission Overdrive Switch',
       '7-Speed Automatic', 'Automatic, 8-Spd PDK Dual-Clutch',
       'Automatic, 8-Spd M STEPTRONIC w/Drivelogic, Sport & Ma

In [35]:
match = re.search(r'(/d+) -Speed', df['transmission'].unique()[0] )
match.group(1)

AttributeError: 'NoneType' object has no attribute 'group'

In [34]:
match

In [37]:
df['transmission'].unique()[0]

'6-Speed A/T'

In [62]:
import pandas as pd
import numpy as np

# Sample dataset
df = pd.read_csv('used_cars.csv')

df['milage'] = df['milage'].str.replace(r'[^\d.]', '', regex=True).astype(float)
df['price'] = df['price'].str.replace(r'[^\d.]', '', regex=True).astype(float)

# Handle missing values
# Fill numerical columns with mean

def extract_hp(engine_desc):
    match = re.search(r'(\d+(\.\d+)?)HP', engine_desc)
    return float(match.group(1)) if match else None

def extract_displacement(engine_desc):
    match = re.search(r'(\d+(\.\d+)?)L', engine_desc)
    return float(match.group(1)) if match else None

def extract_cylinders(engine_desc):
    match = re.search(r'(\d+)\s*Cylinder', engine_desc)
    return int(match.group(1)) if match else None

def extract_fuel_type(engine_desc):
    if 'Gasoline' in engine_desc:
        return 'Gasoline'
    elif 'Diesel' in engine_desc:
        return 'Diesel'
    elif 'Flex Fuel' in engine_desc:
        return 'Flex Fuel'
    else:
        return 'Other'
        
# Extract features from `engine` column
df['hp'] = df['engine'].apply(extract_hp)
df['displacement'] = df['engine'].apply(extract_displacement)
df['cylinders'] = df['engine'].apply(extract_cylinders)
df['fuel_type_engine'] = df['engine'].apply(extract_fuel_type)

# Drop the original `engine` column
df = df.drop('engine', axis=1)

df['milage'] = df['milage'].fillna(df['milage'].mean())

# Fill categorical columns with mode
df['fuel_type'] = df['fuel_type'].fillna(df['fuel_type'].mode()[0])
df['accident'] = df['accident'].fillna(df['accident'].mode()[0])
df['clean_title'] = df['clean_title'].fillna(df['clean_title'].mode()[0])

# One-hot encode categorical columns
categorical_cols = ['brand', 'model', 'fuel_type', 'fuel_type_engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Separate features and target variable
X = df.drop('price', axis=1)
y = df['price']

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Choose a model and train it
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error
y_pred = model.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

# To save the model for later use
import joblib
joblib.dump(model, 'car_price_model.pkl')

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [57]:
df['engine'].unique()

array(['300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability',
       '3.8L V6 24V GDI DOHC', '3.5 Liter DOHC', ...,
       '136.0HP 1.8L 4 Cylinder Engine Gasoline Fuel',
       '270.0HP 2.0L 4 Cylinder Engine Gasoline Fuel',
       '420.0HP 5.9L 12 Cylinder Engine Gasoline Fuel'], dtype=object)

In [63]:
df

,model_year,milage,price,hp,displacement,cylinders,brand_Alfa,brand_Aston,brand_Audi,brand_BMW,...,int_col_Tupelo,int_col_Very Light Cashmere,int_col_WHITE,int_col_Walnut,int_col_Whisper Beige,int_col_White,int_col_White / Brown,int_col_Yellow,int_col_–,accident_None reported
0,2013,51000.0,10300.0,300.0,3.7,6.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2021,34742.0,38005.0,NaN,3.8,NaN,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2022,22372.0,54598.0,NaN,NaN,NaN,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,2015,88900.0,15500.0,354.0,3.5,6.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,2021,9835.0,34999.0,NaN,2.0,NaN,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004,2023,714.0,349950.0,NaN,6.0,NaN,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4005,2022,10900.0,53900.0,349.0,3.0,6.0,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
4006,2022,2116.0,90998.0,NaN,NaN,NaN,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4007,2020,33000.0,62999.0,450.0,3.5,6.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [72]:
df = pd.read_csv('used_cars.csv')

df['milage'] = df['milage'].str.replace(r'[^\d.]', '', regex=True).astype(float)
df['price'] = df['price'].str.replace(r'[^\d.]', '', regex=True).astype(float)

# Handle missing values
# Fill numerical columns with mean

def extract_hp(engine_desc):
    match = re.search(r'(\d+(\.\d+)?)HP', engine_desc)
    return float(match.group(1)) if match else None

def extract_displacement(engine_desc):
    match = re.search(r'(\d+(\.\d+)?)L', engine_desc)
    return float(match.group(1)) if match else None

def extract_cylinders(engine_desc):
    match = re.search(r'(\d+)\s*Cylinder', engine_desc)
    return int(match.group(1)) if match else None

def extract_fuel_type(engine_desc):
    if 'Gasoline' in engine_desc:
        return 'Gasoline'
    elif 'Diesel' in engine_desc:
        return 'Diesel'
    elif 'Flex Fuel' in engine_desc:
        return 'Flex Fuel'
    else:
        return 'Other'
        
# Extract features from `engine` column
df['hp'] = df['engine'].apply(extract_hp)
df['displacement'] = df['engine'].apply(extract_displacement)
df['cylinders'] = df['engine'].apply(extract_cylinders)

# Drop the original `engine` column

df['milage'] = df['milage'].fillna(df['milage'].mean())

# Fill categorical columns with mode
df['fuel_type'] = df['fuel_type'].fillna(df['fuel_type'].mode()[0])
df['accident'] = df['accident'].fillna(df['accident'].mode()[0])
df['clean_title'] = df['clean_title'].fillna(df['clean_title'].mode()[0])

In [73]:
df.head(5)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,hp,displacement,cylinders
0,Ford,Utility Police Interceptor Base,2013,51000.0,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,10300.0,300.0,3.7,6.0
1,Hyundai,Palisade SEL,2021,34742.0,Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,38005.0,NaN,3.8,NaN
2,Lexus,RX 350 RX 350,2022,22372.0,Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,Yes,54598.0,NaN,NaN,NaN
3,INFINITI,Q50 Hybrid Sport,2015,88900.0,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,15500.0,354.0,3.5,6.0
4,Audi,Q3 45 S line Premium Plus,2021,9835.0,Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,Yes,34999.0,NaN,2.0,NaN


In [74]:
df.isnull().sum()

brand             0
model             0
model_year        0
milage            0
fuel_type         0
engine            0
transmission      0
ext_col           0
int_col           0
accident          0
clean_title       0
price             0
hp              808
displacement    377
cylinders       945
dtype: int64

In [103]:
import pandas as pd
import re

# Load dataset
df = pd.read_csv('used_cars.csv')

# Convert `milage` and `price` to numeric, stripping any non-numeric characters
df['milage'] = df['milage'].str.replace(r'[^\d.]', '', regex=True).astype(float)
df['price'] = df['price'].str.replace(r'[^\d.]', '', regex=True).astype(float)

# Define extraction functions
def extract_hp(engine_desc):
    match = re.search(r'(\d+(\.\d+)?)HP', engine_desc)
    return float(match.group(1)) if match else None

def extract_displacement(engine_desc):
    match = re.search(r'(\d+(\.\d+)?)L', engine_desc)
    return float(match.group(1)) if match else None

def extract_cylinders(engine_desc):
    # Match patterns like "V6", "I4", "V8", etc.
    match = re.search(r'\b([VvIi]\d+)\b', engine_desc)
    if match:
        config = match.group(1)
        return int(config[1:])  # Return the number part after V or I

    # Match patterns like "3.5 Liter DOHC"
    match = re.search(r'(\d+(\.\d+)?)\s*L', engine_desc)
    if match:
        displacement = float(match.group(1))
        # Use some heuristic to estimate cylinders based on displacement
        if displacement <= 2.5:
            return 4  # Assume 4 cylinders for engines less than or equal to 2.0 liters
        elif displacement <= 3.8:
            return 6  # Assume 6 cylinders for engines between 2.0 and 3.0 liters
        else:
            return 8  # Assume 8 cylinders for engines larger than 3.0 liters

    # If no pattern matches, return None
    return None

# Extract features from `engine` column
df['hp'] = df['engine'].apply(extract_hp)
df['displacement'] = df['engine'].apply(extract_displacement)
df['cylinders'] = df['engine'].apply(extract_cylinders)

# Impute missing values for `hp`, `displacement`, and `cylinders` with their respective mean values
df['hp'] = df['hp'].fillna(df['hp'].mean())
df['displacement'] = df['displacement'].fillna(df['displacement'].mean())
df['cylinders'] = df['cylinders'].fillna(df['cylinders'].mean())

# Impute missing values for other columns
df['milage'] = df['milage'].fillna(df['milage'].mean())
df['fuel_type'] = df['fuel_type'].fillna(df['fuel_type'].mode()[0])
df['accident'] = df['accident'].fillna(df['accident'].mode()[0])
df['clean_title'] = df['clean_title'].fillna(df['clean_title'].mode()[0])
accident_map = {'At least 1 accident or damage reported': 1, 'None reported': 0}
df['accident'] = df['accident'].replace(accident_map)
df['transmission_type'] = df['transmission'].apply(lambda x: 'Automatic' if 'a/t' in x.lower() or 'automatic' in x.lower() else 'Manual')

df.drop(columns= {'engine', 'transmission'}, axis=1, inplace=True)

df.head(5)


,brand,model,model_year,milage,fuel_type,ext_col,int_col,accident,clean_title,price,hp,displacement,cylinders,transmission_type
0,Ford,Utility Police Interceptor Base,2013,51000.0,E85 Flex Fuel,Black,Black,1,Yes,10300.0,300.000000,3.700000,6.0,Automatic
1,Hyundai,Palisade SEL,2021,34742.0,Gasoline,Moonlight Cloud,Gray,1,Yes,38005.0,332.206498,3.800000,6.0,Automatic
2,Lexus,RX 350 RX 350,2022,22372.0,Gasoline,Blue,Black,0,Yes,54598.0,332.206498,3.709045,6.0,Automatic
3,INFINITI,Q50 Hybrid Sport,2015,88900.0,Hybrid,Black,Black,0,Yes,15500.0,354.000000,3.500000,6.0,Automatic
4,Audi,Q3 45 S line Premium Plus,2021,9835.0,Gasoline,Glacier White Metallic,Black,0,Yes,34999.0,332.206498,2.000000,4.0,Automatic


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brand              4009 non-null   object 
 1   model              4009 non-null   object 
 2   model_year         4009 non-null   int64  
 3   milage             4009 non-null   float64
 4   fuel_type          4009 non-null   object 
 5   ext_col            4009 non-null   object 
 6   int_col            4009 non-null   object 
 7   accident           4009 non-null   int64  
 8   clean_title        4009 non-null   object 
 9   price              4009 non-null   float64
 10  hp                 4009 non-null   float64
 11  displacement       4009 non-null   float64
 12  cylinders          4009 non-null   float64
 13  transmission_type  4009 non-null   object 
dtypes: float64(5), int64(2), object(7)
memory usage: 438.6+ KB


In [106]:
# One-hot encode categorical columns
categorical_cols = ['brand', 'model', 'fuel_type', 'transmission_type', 'ext_col', 'int_col', 'clean_title']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Separate features and target variable
X = df.drop('price', axis=1)
y = df['price']

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Choose a model and train it
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error
y_pred = model.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

# To save the model for later use
import joblib
joblib.dump(model, 'car_price_model.pkl')

MAE: 16674.877967581047
MSE: 18314694548.570057
RMSE: 135331.79430041582


['car_price_model.pkl']